# A Wright-Fisher simulation implemented in C++ via Cython.

This tutorial implements a Wright-Fisher simulation with mutation and recombination using [Cython](http://www.cython.org).  Cython is two things:

* A grammer/dialect of Python that allows static typing Python and of C/C++ types.
* A static compiler to turn the Cython grammer in to C or C++ code to compile into a Python extension module.

Cython has a learning curve of its own. A lot of what is shown below reflects best practices.  For those, we refer you to the Cython documentation.

Here, we avoid all use of [numpy](http://www.numpy.org) until we have to talk to [msprime](http://msprime.readthedocs.io).  We replace all numpy functionality with the equivalent routines from the excellent GNU Scientific Library, or [GSL](https://www.gnu.org/software/gsl/doc/html/index.html). Yes, numpy is fast!  Numpy is written in C!  But, numpy has to talk back and forth to Python, meaning we can out-perform it by writing routines that execute completely on the C side.

This example is closer to reality for those working in lower-level languages.  First, we must build our world, which means defining data types (structs, in this case), functions acting on those types, and a bunch of auxillary code to manage memory and handle errors.  After all that, we can code up the `simplify` and `evolve` functions. Such is the price of speed.

First, we load an extension allowing us to write Cython in a notebook:

In [1]:
%load_ext Cython
# Set ourselves up for some plotting, too
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [51]:
%%cython --cplus --compile-args=-std=c++11 -3 -lgsl -lgslcblas -lm

import msprime
import numpy as np
import struct
cimport cython
cimport numpy as np
from cython.operator cimport dereference as deref
from libc.stdint cimport int32_t, uint32_t
from libcpp.vector cimport vector
from libcpp.unordered_set cimport unordered_set
from libcpp.utility cimport pair
from cython_gsl.gsl_rng cimport *

# Cython doesn't export all of C++'s standard library,
# so we have to expose a few things we need:
cdef extern from "<algorithm>" namespace "std" nogil:
    Iter find[Iter,ValueType](Iter begin, Iter end, const ValueType & value)
    Iter max_element[Iter](Iter begin, Iter end)
    
cdef cppclass TableData:
    # Node data:
    vector[double] time
    # Edge data
    vector[int32_t] parent, child
    vector[double] left, right
    # Mutation data
    vector[double] pos
    vector[int32_t] node, generation
    
    # lookup table to book-keep 
    # infinitely-many sites mutation
    # model
    unordered_set[double] mut_lookup
    
    TableData():
        time.reserve(10000)
        parent.reserve(10000)
        child.reserve(10000)
        left.reserve(10000)
        right.reserve(10000)
        pos.reserve(10000)
        node.reserve(10000)
        generation.reserve(10000)
        
    void clear():
        time.clear()
        parent.clear()
        child.clear()
        left.clear()
        right.clear()
        pos.clear()
        node.clear()
        generation.clear()
        
    @cython.boundscheck(False) # turn off bounds-checking for entire function
    @cython.wraparound(False)  # turn off negative index wrapping for entire function
    void reset_lookup(np.ndarray[double,ndim=1] pos):
        mut_lookup.clear()
        cdef size_t i=0
        for i in range(len(pos)):
            mut_lookup.insert(pos[i])
        
cdef pair[int32_t, int32_t] pick_parents(const gsl_rng *r, const int32_t N, const int32_t first_parental_index):
    cdef pair[int32_t, int32_t] rv
    cdef int32_t p = <int32_t>gsl_ran_flat(r,0.0,<double>N)
    
    rv.first = first_parental_index + 2*p
    rv.second = rv.first+1
    # "Mendel"
    if gsl_rng_uniform(r) < 0.5:
        rv.first, rv.second = rv.second, rv.first
    return rv


cdef void infsites(const gsl_rng *r, const double mu, 
             const int32_t offspring_node,
             const int32_t generation,
             TableData & tables):
    cdef unsigned nmuts=gsl_ran_poisson(r,mu)
    cdef size_t i
    cdef double pos
    for i in range(<size_t>nmuts):
        pos = gsl_rng_uniform(r)
        while tables.mut_lookup.find(pos)!=tables.mut_lookup.end():
            pos = gsl_rng_uniform(r)
        tables.mut_lookup.insert(pos)
        tables.pos.push_back(pos)
        tables.node.push_back(offspring_node)
        tables.generation.push_back(generation)
        
cdef void simplify(TableData & tables, object nodes, object edges,
                   object sites, object mutations,
                   const double dt):
    nodes.set_columns(time=nodes.time+dt,flags=nodes.flags)
    
    # Reverse new node times and add to nodes
    cdef mtime = deref(max_element(tables.time.begin(),tables.time.end()))
    #cdef double mtime = deref(mitr)
    cdef size_t index=0
    for index in range(tables.time.size()):
        tables.time[index]=-1.0*(tables.time[index]-mtime)
    nodes.append_columns(time=np.asarray(<double[:tables.time.size()]>tables.time.data()),
                        flags=np.ones(tables.time.size(),dtype=np.uint32))
    
    edges.append_columns(left=np.asarray(<double[:tables.left.size()]>tables.left.data()),
                        right=np.asarray(<double[:tables.right.size()]>tables.right.data()),
                        parent=np.asarray(<int32_t[:tables.parent.size()]>tables.parent.data()),
                        child=np.asarray(<int32_t[:tables.child.size()]>tables.child.data()))
    for index in range(tables.pos.size()):
        sites.add_row(position=tables.pos[index],
                      ancestral_state=`0`)
        mutations.add_row(site=len(sites)-1,
                          node=tables.node[index],
                         derived_state='1')
        
    
    samples=np.where(nodes.time==0.0)[0]
    msprime.sort_tables(nodes=nodes,edges=edges,sites=sites,
                       mutations=mutations)
    msprime.simplify_tables(samples=samples.tolist(),
                           nodes=nodes,
                           edges=edges,
                           sites=sites,
                           mutations=mutations)
                            
    tables.clear()
    
    
def evolve(int N, int ngens, double theta, double rho, int gc, int seed):
    nodes = msprime.NodeTable()
    nodes.set_columns(time=np.zeros(2*N),flags=np.ones(2*N,dtype=np.uint32))
    edges = msprime.EdgeTable()
    sites = msprime.SiteTable()
    mutations = msprime.MutationTable()
    
    
    cdef double mu = theta/(4*<double>N)
    cdef TableData tables
    cdef gsl_rng * r = gsl_rng_alloc(gsl_rng_mt19937)
    gsl_rng_set(r,seed)
    cdef size_t generation=0,offspring=0
    cdef int32_t next_offspring_index = 2*N #Same as len(nodes)
    cdef int32_t first_parental_index = 0
    
    for generation in range(ngens):
        for offspring in range(N):
            parents1 = pick_parents(r,N,first_parental_index)
            parents2 = pick_parents(r,N,first_parental_index)
            tables.time.push_back(generation+1.0)
            tables.time.push_back(generation+1.0)
            
            tables.left.push_back(0.0)
            tables.right.push_back(1.0)
            tables.parent.push_back(parents1.first)
            tables.child.push_back(next_offspring_index)
            infsites(r,mu,next_offspring_index,generation+1,tables)
            
            tables.left.push_back(0.0)
            tables.right.push_back(1.0)
            tables.parent.push_back(parents2.first)
            tables.child.push_back(next_offspring_index+1)
            infsites(r,mu,next_offspring_index+1,generation+1,tables)
            next_offspring_index += 2
        first_parental_index = next_offspring_index - 2*N
        
    print(tables.pos.size(),tables.mut_lookup.size())
    simplify(tables,nodes,edges,sites,mutations,ngens)
    gsl_rng_free(r)
    
    return msprime.load_tables(nodes=nodes,edges=edges,sites=sites,mutations=mutations)

In [52]:
%%time
evolve(1000,20000,100,0,0,42)

1000008 1000008
CPU times: user 16.7 s, sys: 4.22 s, total: 21 s
Wall time: 21 s


# Comparison to msprime

In this section, we compare the distribution of outputs to msprime using [pylibseq](https://github.com/molpopgen/pylibseq), a Python interface to [libsequence](http://molpopgen.github.io/libsequence/)

In [53]:
from IPython.display import SVG
import msprime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from libsequence.polytable import SimData
from libsequence.summstats import PolySIM
from libsequence.msprime import make_SimData
import concurrent.futures
import pandas as pd
from collections import namedtuple

SummStats=namedtuple('SummStats',['S','pi','D','hprime','rmin'])

Let's take a quick tour of pylibseq:

In [54]:
# Simulate data with msprime
ts = msprime.simulate(10,mutation_rate=1,random_seed=666)

# Get it into the format expected by pylibseq
d = make_SimData(ts)

# This should look familiar! :)
print(d)

//
segsites: 10
positions: 0.26833 0.290974 0.325691 0.349762 0.369596 0.390163 0.583813 0.743047 0.767412 0.978347
0100100100
1000010000
0010001000
0100100100
0001100100
0000100101
0000100110
1000010000
1000010000
0100100100


In [55]:
# Create object to calculate summary stats
x = PolySIM(d)
# Calculate a few:
print(x.thetapi(),x.tajimasd(),x.hprime(),x.rm())

3.4666666666666672 -0.08550572842849505 0.5368222463065044 0


In [56]:
%%time
msprime_raw_data=[]
for i in msprime.simulate(10,mutation_rate=100.0/4.0,
                          recombination_rate=0.0,
                          num_replicates=1000,
                          random_seed=42):
    d = make_SimData(i)
    ps = PolySIM(d)
    # A little check that the two pieces of code agree
    assert(ps.numpoly() == i.num_mutations)
    msprime_raw_data.append(SummStats(ps.numpoly(),
                                      ps.thetapi(),ps.tajimasd(),
                                      ps.hprime(),ps.rm()))

CPU times: user 5.2 s, sys: 11.6 ms, total: 5.21 s
Wall time: 5.23 s


To run the forward simulations, we will use multiple Python processes via Python 3's [`concurrent.futures`](https://docs.python.org/3/library/concurrent.futures.html) library. The short of it is that we need a Python function to send out to different processes and return results, which will be pickled into a future back in the main process.

In [57]:
def run_forward_sim(nreps,seed,repid):
    """
    Run our forward sim, calculate
    a bunch of stats, and return 
    the list.
    """
    # Not the best seeding scheme, 
    # but good enough for now...
    np.random.seed(seed)
    msp_rng = msprime.RandomGenerator(int(seed))
    seeds = np.random.randint(0,1000000,nreps)
    sims = []
    for i in range(nreps):
        # print('inputs:',repid,seeds[i])
        ts = evolve(500,10000,100.0,0.0,500,seeds[i])
        samples = np.random.choice(1000,10,replace=False)
        assert(all(ts.tables.nodes.time[samples]==0.0))
        ts2 = ts.simplify(samples=samples.tolist())
        # n=msprime.NodeTable()
        # e=msprime.EdgeTable()
        # s=msprime.SiteTable()
        # m=msprime.MutationTable()
        # ts2.dump_tables(nodes=n,edges=e)
        # print(n.time.max())
        # mutgen = msprime.MutationGenerator(
        #     msp_rng, 100.0/(float(4*500)))
        # mutgen.generate(n,e,s,m)
        # ts2=msprime.load_tables(nodes=n,edges=e,sites=s,mutations=m)
        # print(samples)
        # print(n.time[samples])
        # print(s)
        # print(m)
        # Simplify from entire pop down
        # to random sample of n << 2N
        # slist = samples.tolist()
        # slist.append(8)
        # ts2=ts.simplify(slist)
        # print(ts2.num_mutations)
        # print(len(ts2.tables.nodes),
        #      len(ts2.tables.edges),
        #      len(ts2.tables.sites),
        #      len(ts2.tables.mutations))
        d=make_SimData(ts2)
        ps=PolySIM(d)
        sims.append(SummStats(ps.numpoly(),
                              ps.thetapi(),
                              ps.tajimasd(),
                              ps.hprime(),
                              ps.rm()))
    return sims

In [58]:
%%time
x=run_forward_sim(1,66,3511)
print(x)

499259 499259
[SummStats(S=525, pi=214.31111111111005, D=0.7758200022046403, hprime=0.6332319801973835, rmin=0)]
CPU times: user 4.8 s, sys: 1.12 s, total: 5.92 s
Wall time: 5.91 s


In the next bit, we map our function into four separate processes.

**Note:** We could use a `concurrent.futures.ThreadPoolExecutor` instead of the process pool executor.  However, some of our Cython functions rely on Python types (`list`, `dict`, etc.), meaning that the Global Interpreter Lock is a barrier to efficient concurrency.  In practice, we've found it better to take the hit of pickling between processes so that your simulations can run at 100% CPU in different processes.

In [ ]:
%%time
fwd_sim_data=[]
np.random.seed(16463623)
with concurrent.futures.ProcessPoolExecutor(max_workers=4) as executor:
    futures = {executor.submit(run_forward_sim,50,np.random.randint(0,2000000,1)[0],i): i for i in range(4)}
    for fut in concurrent.futures.as_completed(futures):
        fn = fut.result()
        fwd_sim_data.extend(fn)

500497 500497
498934 498934
500517 500517
500799 500799
500257 500257
499961 499961
499775 499775
499155 499155
498239 498239
498899 498899
499763 499763
498240 498240
499459 499459
499648 499648
499439 499439
498961 498961
500314 500314
498829 498829
499518 499518
500138 500138
500281 500281
499355 499355
498454 498454
499630 499630
500590 500590
499668 499668
498875 498875
500088 500088
500454 500454
500876 500876
500486 500486
500072 500072
499828 499828
500521 500521
501162 501162
500808 500808
500386 500386
500637 500637
501606 501606
499688 499688
499794 499794
501109 501109
499888 499888
500332 500332
499586 499586
499150 499150
499929 499929
499360 499360
500265 500265
501072 501072
499559 499559
500453 500453
500212 500212
500977 500977
501143 501143
500878 500878
500392 500392
499591 499591
499428 499428
499370 499370


In [ ]:
msprime_df = pd.DataFrame(msprime_raw_data)
msprime_df['engine'] = ['msprime']*len(msprime_df.index)
fwd_df = pd.DataFrame(fwd_sim_data)
fwd_df['engine']=['forward']*len(fwd_df)
summstats_df = pd.concat([msprime_df,fwd_df])

In [ ]:
sns.set(style="darkgrid")
g = sns.FacetGrid(summstats_df,col="engine",margin_titles=True)
bins = np.linspace(summstats_df.pi.min(),summstats_df.pi.max(),20)
g.map(plt.hist,'pi',bins=bins,color="steelblue",lw=0,normed=True);

g = sns.FacetGrid(summstats_df,col="engine",margin_titles=True)
bins = np.linspace(summstats_df.S.min(),summstats_df.S.max(),20)
g.map(plt.hist,'S',bins=bins,color="steelblue",lw=0,normed=True);

g = sns.FacetGrid(summstats_df,col="engine",margin_titles=True)
bins = np.linspace(summstats_df.D.min(),summstats_df.D.max(),20)
g.map(plt.hist,'D',bins=bins,color="steelblue",lw=0,normed=True);

g = sns.FacetGrid(summstats_df,col="engine",margin_titles=True)
bins = np.linspace(summstats_df.rmin.min(),summstats_df.rmin.max(),20)
g.map(plt.hist,'rmin',bins=bins,color="steelblue",lw=0,normed=True);

In [ ]:
len(fwd_df.index)

In [ ]:
from scipy.stats import ks_2samp

In [ ]:
print(summstats_df.groupby(['engine']).agg(['mean','std']))

The stats are clearly off:

In [ ]:
ks_2samp(fwd_df.pi,msprime_df.pi)

In [ ]:
ks_2samp(fwd_df.S,msprime_df.S)

In [ ]:
ks_2samp(fwd_df.D,msprime_df.D)

In [ ]:
ks_2samp(fwd_df.rmin,msprime_df.rmin)